In [13]:
# Part 2(1). calculate signatures and make figures.
# Author: H., Liu, Nov.03, 2020.

import numpy as np
import os,datetime
import pandas as pd
import xarray as xr

def read_obs(file, var):
    '''read obsrved streamflow.
    file is the file path of observed streamflow.
    var is the string representing streamflow in the file.'''
    f        = xr.open_dataset(file)
    time     = pd.to_datetime(f['time'].values[:])
    varValue = f.variables[var].values[:]        
    return time,varValue

def read_sim(file, var):    
    '''read summa simulated streamflow.
    file is the file path of simulated streamflow.
    var is the string representing streamflow in the file.'''
    f        = xr.open_dataset(file)
    time     = pd.to_datetime(f['time'].values[:])
    varValue = f.variables[var].values[:]  
    return time,varValue

def read_attr(file):
    '''read summa attribute file.
    file is the attribtue file path.'''
    f         = xr.open_dataset(file)
    hruId     = f['hruId'].values[:]
    latitude  = f.variables['latitude'].values[:]        
    longitude = f.variables['longitude'].values[:]  
    elevation = f.variables['elevation'].values[:] 
    return hruId,latitude,longitude,elevation

def calculate_freq(time, flow, threshold, freq_type):
    ''' calculate high/low flow frequency.
    time is a time series in the format of datetime.
    flow is an array of the flow data in shape of (time,usgs_id).
    threshold is an array of the high/low flow threshold in shape of (usgs_id).
    freq_type is a string selected from "high" and "low". 
    "high" means calcualting the highflow frequency."low" means calcualting the lowflow frequency.""'''
        
    # reshape threshold to be the same as the flow array
    ntimes    = np.shape(flow)[0]
    threshold = np.expand_dims(threshold, axis=0)    # shape(1,usgs_id)
    threshold = np.repeat(threshold, ntimes, axis=0) # shape(time,usgs_id)
    
    # high/low condition check
    if freq_type == 'high':
        satisfy_mask = np.where(flow>threshold,1,0)
    elif freq_type == 'low':
        satisfy_mask = np.where(flow<threshold,1,0)
    
    # calculate frequency in unit days/year
    df = pd.DataFrame(satisfy_mask,index=time)
    df_yr = df.groupby(df.index.year).sum()
    df_mean = df_yr.mean(axis=0)    
    return df_mean.values

#====================================================================================#
# Main script

# specify directories and files
rootDir  = '/Users/hongli/Documents/course/assign/assign5'
obsFile  = os.path.join(rootDir, '5_obs/hcdn_obs_depth.nc')
simFile  = os.path.join(rootDir, 'output/camels_nldas_full.nc')
attrFile = os.path.join(rootDir, 'settings/local_attribute_camels.nc')

time_format    = '%Y-%m-%d'
date_start     = '1980-10-01'
date_end       = '2008-09-30'
date_start_obj = datetime.datetime.strptime(date_start, time_format)
date_end_obj   = datetime.datetime.strptime(date_end, time_format)

outDir = os.path.join(rootDir,'assign_output/2.2_signature')
if not os.path.exists(outDir):
    os.makedirs(outDir)

# --- read attribute data --- 
hruId,latitude,longitude,elevation = read_attr(attrFile)

# --- calculate flow signature ---
# loop through observed and simulated flows
for i in range(1):

    # read observed/simulated flow
    if i == 0:
        time, data = read_obs(obsFile, 'qobs')
        ofile = os.path.join(outDir,'2.2.1_obs.signature.txt')
    elif i == 1:
        time, data = read_sim(simFile, 'averageRoutedRunoff_mean')
        ofile = os.path.join(outDir,'2.2.2_sim.signature.txt')
        
    # extract data for the simulation time period
    time_mask    = (time >= date_start_obj) & (time <= date_end_obj) 
    time         = time[time_mask]                
    data         = data[time_mask,:]*1000*24*3600 # convert unit (m/s -> mm/day)
    data[data<0] = 0.0                            # revise negative flows
    
    # calculate three signatures
    # (1) mean daily discharge. shape(usgs_id)
    q_mean   = np.nanmean(data,axis=0) 
    q_median = np.nanmedian(data,axis=0) 

    # (2) frequency of high-flow days ( > 9 times the median daily flow)
    high_q_freq_threshold = q_median*9.0
    high_q_freq           = calculate_freq(time, data, high_q_freq_threshold, 'high')

    # (3) frequency of low-flow days ( < 0.2 times the mean daily flow)
    low_q_freq_threshold = q_mean*0.2
    low_q_freq           = calculate_freq(time, data, low_q_freq_threshold, 'low')
    
    # save results
    df = pd.DataFrame({'hruId': hruId, 'latitude': latitude, 'longitude':longitude, 'elevation':elevation,
                       'q_mean':q_mean, 'high_q_freq':high_q_freq, 'low_q_freq':low_q_freq})
    df.to_csv(ofile, sep=',', index=False)
            
print('Done')

/Users/hongli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:85: RuntimeWarning: invalid value encountered in less
/Users/hongli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in greater


Done


/Users/hongli/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in less


In [1]:
# Part 2(2). plot signatures.
import os
os.environ["PROJ_LIB"] = '/glade/u/home/hongli/tools/miniconda3/envs/conda_hongli/share/proj'
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt 
import numpy as np
import os,datetime
import pandas as pd
import xarray as xr

def plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax):
''' plot basemap using the Equidistant Cylindrical projection.
    - llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat are the lat/lon values of the lower left and upper right corners of the map.
    - ax is the axis to plot basemap.'''
    
    m = Basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,resolution='l',projection='cyl', ax=ax)   
    m.drawstates(linewidth=1, linestyle='solid', color='grey')
    m.drawcountries(linewidth=1, linestyle='solid', color='k')
    m.drawcoastlines(linewidth=.75, linestyle='solid', color='k')

    # add latitude and longitude
    m.drawparallels(np.arange(np.floor(llcrnrlat),np.ceil(urcrnrlat),10),labels=[True,False,False,False],
                    dashes=[1,1], fontsize='xx-small', linewidth=0.2, color='grey') # Draw parallels (latitude lines)
    m.drawmeridians(np.arange(np.floor(llcrnrlon),np.ceil(urcrnrlon),15),labels=[False,False,False,True],
                    dashes=[1,1], fontsize='xx-small', linewidth=0.2, color='grey') # Draw meridians (longitude lines). Label [left, right, top, bottom]    

    return m

def plot_sig(latitude,longitude,sig_value,sig_name,sig_unit,title,cmap,vmin,vmax,oFile):
''' plot each signature.
    - latitude,longitue, and sig_value are input arrays, read from signature output text.
    - sig_name and sig_unit are input strings, used to define colorbar label.
    - title is input string, used as the figure title.
    - cmap is input string, used as the color map.
    - vmin and vmax are input numeric values, used as the lower and upper limits of colorbar range.
    - oFile is the path for saving the plot figure. '''

    fig, ax = plt.subplots(1, 1)
    llcrnrlon = np.min(longitude)
    urcrnrlon = np.max(longitude)
    llcrnrlat = np.min(latitude)
    urcrnrlat = np.max(latitude)

    # plot Basemap
    m = plot_basemap(llcrnrlon,llcrnrlat,urcrnrlon,urcrnrlat,ax)          

    # plot signature
    sig = m.scatter(longitude, latitude, latlon=True, c=sig_value, cmap=cmap, vmin=vmin,vmax=vmax)

    # add title
    ax.set_title(title, fontsize='small', fontweight='semibold')

    # plot colorbar    
    fig.subplots_adjust(bottom=0.1, top=1, left = 0, right=0.9, wspace = 0.05, hspace = 0.05)
    cax  = fig.add_axes([0.93, 0.28, 0.05, 0.55]) 
    cbar = fig.colorbar(sig, cax=cax)
    cbar.ax.tick_params(labelsize='small')
    
    # set the colorbar ticks and tick labels
    cbar.set_label(label=sig_name+' ['+sig_unit+']',size='small')    

    # save plot
    fig.savefig(oFile, dpi=150, bbox_inches = 'tight', pad_inches = 0.05)
    plt.close(fig)
    return

# specify directories and files
rootDir  = '/glade/u/home/hongli/work/course/assign5/pbhmCourse_student/5_high_performance_computing/'
outDir = os.path.join(rootDir,'assign_output/2.2_signature')
if not os.path.exists(outDir):
    os.makedirs(outDir)

# read signatures obtained from Part1
obs_sig = np.loadtxt(os.path.join(outDir,'2.2.1_obs.signature.txt'), delimiter=',', 
                     usecols=[1,2,4,5,6],skiprows=1) # read five columns: latitude, longitude, three signatures.
sim_sig = np.loadtxt(os.path.join(outDir,'2.2.2_sim.signature.txt'), delimiter=',', 
                     usecols=[-3,-2,-1],skiprows=1) # read three columns: three signatures.
sig     = np.concatenate((obs_sig,sim_sig), axis=1)

# get the min and max values of three signatures, used as colobar range.
q_mean_vmin = np.nanmin(sig[:,[2,5]])   
q_mean_vmax = np.nanmax(sig[:,[2,5]]) 

high_q_freq_vmin = np.nanmin(sig[:,[3,6]])    
high_q_freq_vmax = np.nanmax(sig[:,[3,6]])

low_q_freq_vmin = np.nanmin(sig[:,[4,7]])   
low_q_freq_vmax = np.nanmax(sig[:,[4,7]])  

# plot three signatures
# q_mean
plot_sig(latitude=sig[:,0],longitude=sig[:,1],sig_value=sig[:,2],sig_name='q_mean',\
         sig_unit='mm/day', title='Observed mean daily discharge', \
         cmap='Blues', vmin=q_mean_vmin, vmax=q_mean_vmax, oFile=os.path.join(outDir,'q_mean_obs.png'))
plot_sig(latitude=sig[:,0],longitude=sig[:,1],sig_value=sig[:,5],sig_name='q_mean',\
         sig_unit='mm/day', title='Simulated mean daily discharge', \
         cmap='Blues', vmin=q_mean_vmin, vmax=q_mean_vmax, oFile=os.path.join(outDir,'q_mean_sim.png'))

# high_q_freq
plot_sig(latitude=sig[:,0],longitude=sig[:,1],sig_value=sig[:,3],sig_name='high_q_freq',\
         sig_unit='-', title='Observed frequency of high-flow days', \
         cmap='Reds', vmin=high_q_freq_vmin, vmax=high_q_freq_vmax, oFile=os.path.join(outDir,'high_q_freq_obs.png'))
plot_sig(latitude=sig[:,0],longitude=sig[:,1],sig_value=sig[:,6],sig_name='high_q_freq',\
         sig_unit='-', title='Simulated frequency of high-flow days', \
         cmap='Reds', vmin=high_q_freq_vmin, vmax=high_q_freq_vmax, oFile=os.path.join(outDir,'high_q_freq_sim.png'))

# low_q_freq
plot_sig(latitude=sig[:,0],longitude=sig[:,1],sig_value=sig[:,4],sig_name='low_q_freq',\
         sig_unit='-', title='Observed frequency of low-flow days', \
         cmap='Greens', vmin=low_q_freq_vmin, vmax=low_q_freq_vmax, oFile=os.path.join(outDir,'low_q_freq_obs.png'))
plot_sig(latitude=sig[:,0],longitude=sig[:,1],sig_value=sig[:,7],sig_name='low_q_freq',\
         sig_unit='-', title='Simulated frequency of low-flow days', \
         cmap='Greens', vmin=low_q_freq_vmin, vmax=low_q_freq_vmax, oFile=os.path.join(outDir,'low_q_freq_sim.png'))


ModuleNotFoundError: No module named 'mpl_toolkits.basemap'